*Copyright 2019 StarkWare Industries Ltd.<br> Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.starkware.co/open-source-license/ <br> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.*

# Part 1: 执行轨迹和低度扩展
- [Video Lecture (youtube)](https://www.youtube.com/watch?v=Y0uJz9VL3Fo)
- [Slides (PDF)](https://starkware.co/wp-content/uploads/2021/12/STARK101-Part1.pdf)

今天我们将为在一个有限域上的FibonacciSq序列开发一个STARK证明系统。
FibonacciSq序列的计算公式为 $a_{n+2} = a_{n+1} ^2 + a_n ^2$
<br>
在今天的代码中，你的代码将会生成一个*STARK*证明，证明:<br>**你知道一个$X\in \mathbb{F}$，当计算一个前两个元素为[$1,X$]的FibonacciSq序列时，第1023个元素的结果为$2338775057$。**
<br><br>
## 基础知识
### Fieldlement 类
我们使用`FieldElement`类来表示有限域元素。<br>你可以从整数创建`FieldElement`的实例，然后进行加法、乘法、除法、求逆等操作。
这个类的底层是$\mathbb{F}_{3221225473}$ ($3221225473 = 3 \cdot 2^{30} + 1$)，所以所有的操作都是模3221225473进行的。
<br><br>
尝试运行一下这段代码 (shift + enter):

In [ ]:
from field import FieldElement
FieldElement(3221225472) + FieldElement(10)

## FibonacciSq 执行轨迹

首先，构建一个长度为1023的数组 `a`，其前两个元素为1和3141592，接下来的1021个元素为由这两个元素生成的FibonacciSq序列。`a`被称为FibonacciSq的执行轨迹，或者当上下文明确时，也可以简称轨迹。
请完善一下代码，填充`a`:

In [ ]:
a = [FieldElement(1), FieldElement(3141592)]
'YOUR_CODE_HERE'

答案 (点击 ... 展开):

In [ ]:
a = [FieldElement(1), FieldElement(3141592)]
while len(a) < 1023:
    a.append(a[-2] * a[-2] + a[-1] * a[-1])

### 测试你的代码
运行下一段代码，检查你的代码是否正确填充了`a`。<br> 注意，这段代码实际上是一个验证器，虽然它非常简单和非简洁。它会遍历整个序列，逐个检查元素是否正确。

In [ ]:
assert len(a) == 1023, '执行轨迹必须只有1023个元素'
assert a[0] == FieldElement(1), '执行轨迹的第一个元素必须是1'
for i in range(2, 1023):
    assert a[i] == a[i - 1] * a[i - 1] + a[i - 2] * a[i - 2], f'第{i}个元素不对'
assert a[1022] == FieldElement(2338775057), '最后一个元素不对'
print('成功🏅!')

## 多项式化
我们现在要将序列视为某个尚未确定的1022次多项式 $f$ 在特定域上的求值（根据Unisolvence Theorem）。
因为后续的一些原因，我吗选择了 $G \subseteq \mathbb{F}^\times$ 的子群，大小为1024。

（回顾一下，$\mathbb{F}^\times$ 表示 $\mathbb{F}$ 的乘法群，它是从 $\mathbb{F}$ 中去掉零元素后得到的。$\mathbb{F}^\times$ 是一个循环群，大小为 $3\cdot 2^{30}$，所以它包含一个大小为 $2^i$ 的子群，其中 $0 \leq i \leq 30$。）
### 寻找阶为1024的群
（注：群的阶和群的元素个数是同一个概念。即，群的size和order是同一个东西。）

（注：元素的阶指的是元素a的阶是使得 $ a^n = e $ 的最小正整数n,其中e是群的单位元）

如果我们找到了一个元素 $g \in \mathbb{F}$ 的阶为1024，那么 $g$ 将生成一个阶为1024的群 $G$。
`FieldElement` 类提供了静态方法 `generator()`，它返回一个生成 $\mathbb{F}^\times$ 的元素（$\mathbb{F}^\times$ 的阶为 $|\mathbb{F}^\times|$）。
1. 使用它来获取一个生成 $G$ 的元素 $g$。
2. 创建一个列表 `G`，包含 $G$ 的所有元素，使得 $G[i] := g^i$。

*提示：当 $k$ 整除 $|\mathbb{F}^\times|$ 时，$g^k$ 生成一个阶为 $\frac {|\mathbb{F}^\times|}{k}$ 的群，而 `FieldElement` $x$ 的 $n$ 次方可以通过调用 `x ** n` 来计算。*


In [ ]:
# 将下面的代码改成使得 g 生成一个阶为 1024 的群
g = FieldElement.generator()
# 创建一个列表 G，包含 G 的所有元素，使得 G[i] := g^i
G = [] 

答案:

In [ ]:
g = FieldElement.generator() ** (3 * 2 ** 20)
G = [g ** i for i in range(1024)]

运行下面这段代码来测试你的代码

In [ ]:
# 检查 g 和 G 是否正确
assert g.is_order(1024), 'g 的阶不对'
b = FieldElement(1)
for i in range(1023):
    assert b == G[i], 'G 的第 i 个元素不对'
    b = b * g
    assert b != FieldElement(1), f'g 的阶大于 {i + 1}'
    
if b * g == FieldElement(1):
    print('成功')
else:
    print('g 的阶大于 1024')

### 多项式 类
现在这里有一个类 `Polynomial`，构造一个多项式 `Polynomial` 的最简单的方式是使用变量 `X`（注意，它是大写的 `X`），它代表多项式中的变量 $x$：

In [ ]:
from polynomial import X
# 多项式 2x^2 + 1
p = 2*X**2 + 1
# 计算 p 在 2 处的值
print(p(2))
# 在最后一行输出多项式
p

### 多项式插值(Interpolating a Polynomial) 
`polynomial` 模块提供了拉格朗日插值函数，它的参数是：
* x_values: 多项式在有限域G中的已知的x坐标。[列表]
* y_values: 对应的y坐标。[列表]

该函数返回一个`Polynomail` 度数小于`len(x_values)`，它在`x_values[i]`处对应的值为`y_values[i]`。

(注：度数是多项式的最高次项的指数，比如 $ x^4 + 3x^2 $ 的度数为4)

运行下面的代码来获取函数`interpolate_poly`的调用帮助信息。

In [ ]:
from polynomial import interpolate_poly
interpolate_poly?

假设 a 包含了某个定义在有限域 G 上的多项式的取值（除了 G[-1]，因为 a 的长度比 G 少一个元素）。
使用`interpolate_poly()` 函数来构造多项式`f`，然后获取在点 `FieldElement(2)` 的值。

In [ ]:
from polynomial import interpolate_poly
# 完善代码，使得 v 等于 f 在点 FieldElement(2) 处的值
# 注意：`interpolate_poly` 可能需要长达一分钟才能运行完成。
v = 'YOUR_CODE_HERE'

Solution:

In [ ]:
f = interpolate_poly(G[:-1], a)
v = f(2)

Run test:

In [ ]:
assert v == FieldElement(1302089273)
print('成功!')

## 在更大的域上求值
我们可以将轨迹视为$f$在$G$上的求值，然后通过在更大的域上求值$f$来扩展它，从而构造出里德-所罗门（Reed-Solomon）纠错码。

### 陪集
为此,我们需要确定一个更大的域来对 $f$ 进行求值。

我们将使用一个比 $G$ 大8倍的域。<br> 一个理所当然的选择是取一个大小为8192的陪集 $H$（这个群存在，因为8192整除 $|\mathbb{F}^\times|$ ）。然后，用 $H$ 中的每个元素乘以 $\mathbb{F}^\times$ 的生成元，从而得到一个新的[陪集](https://en.wikipedia.org/wiki/Coset) $H$。

创建一个列表 `H` ，包含 $H$ 的所有元素，然后将每个元素乘以 $\mathbb{F}^\times$ 的生成元,得到一个名为 `eval_domain` 的列表。换句话说,eval_domain = $\{w\cdot h^i | 0 \leq i <8192 \}$,其中 $h$ 是 $H$ 的生成元, $w$ 是 $\mathbb{F}^\times$ 的生成元。

提示：你已经知道如何得到 $H$ 了，同样的道理，你也知道如何得到 $G$ 。


In [ ]:
# 修正以下内容，确保 H 中的元素按顺序是其生成元的幂
# 也就是说 - H[0] 将是单位元，H[1] 将是 h（H 的生成元），H[2] 将是 H 的生成元的平方，依此类推
H = 'YOUR_CODE_HERE'
eval_domain = 'YOUR_CODE_HERE'

答案:

In [ ]:
w = FieldElement.generator()
h = w ** ((2 ** 30 * 3) // 8192)
H = [h ** i for i in range(8192)]
eval_domain = [w * x for x in H]


运行测试:

In [ ]:
from hashlib import sha256
assert len(set(eval_domain)) == len(eval_domain)
w = FieldElement.generator()
w_inv = w.inverse()
assert '55fe9505f35b6d77660537f6541d441ec1bd919d03901210384c6aa1da2682ce' == sha256(str(H[1]).encode()).hexdigest(),\
    'H list is incorrect. H[1] should be h (i.e., the generator of H).'
for i in range(8192):
    assert ((w_inv * eval_domain[1]) ** i) * w == eval_domain[i]
print('Success!')

### 在陪集上求值
是时候使用`interpolate_poly`和`Polynomial.poly`来在陪集上求值了。请注意，在我们的Python模块中，这个操作实现得很简单，所以插值可能需要一分钟的时间。<br>
实际上，虽然可以使用更高效的算法（例如FFT），但对计算轨迹多项式插值和在陪集上求值的仍然是STARK协议中最耗时的步骤。

In [ ]:
# Fill f_eval with the evaluations of f on eval_domain.
# 完善下面的代码，使得 f_eval 为 使用 f 在 `eval_domain` 中的所有元素的运算结果。
f_eval = 'YOUR_CODE_HERE'

答案:

In [ ]:
f = interpolate_poly(G[:-1], a)
f_eval = [f(d) for d in eval_domain]

运行测试:

In [ ]:
# 根据预先计算的哈希值进行测试
from hashlib import sha256
from channel import serialize
assert '1d357f674c27194715d1440f6a166e30855550cb8cb8efeb72827f6a1bf9b5bb' == sha256(serialize(f_eval).encode()).hexdigest()
print('Success!')

## 承诺
我们将会使用基于[Sha256](https://en.wikipedia.org/wiki/SHA-2)的[Merkle Trees](https://en.wikipedia.org/wiki/Merkle_tree)作为我们的承诺方案。
`MerkleTree` 类提供了一个简单的实现。
请运行下一段代码（在本教程中，这也是对整个代码的正确性计算测试）

In [ ]:
from merkle import MerkleTree
f_merkle = MerkleTree(f_eval)
assert f_merkle.root == '6c266a104eeaceae93c14ad799ce595ec8c2764359d7ad1b4b7c57a4da52be04'
print('Success!')

## 通道(Channel)
理论上来说，STARK证明系统是一个证明者和验证者之间的交互协议。但是在实践中，我们使用[Fiat-Shamir Heuristic](https://en.wikipedia.org/wiki/Fiat%E2%80%93Shamir_heuristic)将这个交互协议转换为非交互证明。在本教程中，你将使用`Channel`类，它实现了这个转换。这个通道的意思是，证明者（你正在编写的）会发送数据，接收随机数或者随机`FieldElement`实例。

这段简单的代码实例化了一个通道对象，并且发送了你的Merkle树的根，之后，可以调用这个通道对象来提供随机数或者随机`FieldElement`实例。

In [ ]:
from channel import Channel
channel = Channel()
channel.send(f_merkle.root)

最后，你可以通过打印`Channel.proof`来获取到通道中传递的数据（即，目前所有通过通道发送的内容）。

In [ ]:
print(channel.proof)